## Imports

In [1]:
import fiona
import pandas as pd
import geopandas as gpd
import folium
import os

## PD Set Options

In [78]:
pd.set_option('display.max_columns',None)

## Links:

### County

Query link: https://gis-mdc.opendata.arcgis.com/search?q=permits

Permits up to two years prior: https://gis-mdc.opendata.arcgis.com/datasets/MDC::building-permits-issued-by-miami-dade-county-2-previous-years-to-present/explore

CURRENT: Permits up to three years prior (with point-feature class): https://gis-mdc.opendata.arcgis.com/datasets/MDC::building-permit/explore?location=25.620364%2C-80.404078%2C10.28

Permit classification table: https://gis-mdc.opendata.arcgis.com/datasets/MDC::tbl-bldg-permit-prop-use/explore

### City
Current City of Miami Permit Data: https://datahub-miamigis.opendata.arcgis.com/datasets/MiamiGIS::building-permits-since-2014/about

Miami City department of buildings directory: https://www.miamigov.com/My-Government/Departments/Building/Building-Department-Directory

iBuild: https://apps.miamigov.com/iBuildPortal/

DEPRECIATED DATA: City permit data: https://data.miamigov.com/Government/Building-Permits-Issued-By-City-Of-Miami-From-2014/7ey5-m434

### TRD Internal

Troubled dev spreadsheet: https://docs.google.com/spreadsheets/d/10d81uPSFiMHXIoCPQC-aJxm__WarYsulVC8H4R7OCSc/edit#gid=0

## Read in City Data

In [4]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
greater_dtown_miami = gpd.read_file('Greater Downtown Miami.kml', driver = 'KML')

In [5]:
miami_city_permits = gpd.read_file('Building_Permits_Since_2014.geojson')

## Filter to only Greater Miami Area

In [6]:
# Perform a spatial join
joined_permits = gpd.sjoin(miami_city_permits, greater_dtown_miami, how='inner', op='within')

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [51]:
miami_city_permits['BuildingPermitStatusDescription'].value_counts()

Final      101188
Active      18551
Expired      4045
Revoked      2872
Hold          364
Name: BuildingPermitStatusDescription, dtype: int64

In [7]:
print(len(miami_city_permits))
print('-------')
print(len(joined_permits))

127020
-------
39510


In [58]:
joined_permits_final = joined_permits.loc[(joined_permits['ScopeofWork'] == 'NEW CONSTRUCTION') | (joined_permits['ScopeofWork'] == 'PHASED PERMIT')]

In [59]:
joined_permits_final = joined_permits_final.loc[joined_permits_final['IssuedDate'] > '2016-12-31']

In [61]:
joined_permits_final_resi = joined_permits_final.loc[joined_permits_final['PropertyType'] == 'Residential']

In [63]:
joined_permits_final_resi.to_csv('ResiPermitsSince2017.csv')

In [57]:
joined_permits_final.to_csv('GreatDtownMiamiPermitsSince2017ALL.csv')

In [46]:
joined_permits_final = joined_permits_final.sort_values(by='TotalSQFT', ascending=False).head(100)

## Explore Data

In [65]:
'01-0108-000-1015'
'101080001015' ## 400 Biscayne Blvd
'101020301010' ## Can't find
'102050301090' ## 99 Southwest 7 St


'01-0108-000-1015'

In [87]:
miami_city_permits.loc[miami_city_permits['FolioNumber'] == 102050301090]

,ID,AdditionSQFT,ApplicationNumber,BuildingFinalLastInspDate,BuildingFinalLastInspResult,BuildingPermitStatusDescription,BuildingPermitStatusReasonDescr,Certificatecode,Certificatedate,CompanyAddress,CompanyCity,CompanyName,CompanyZip,DaysInCity,DaysInCityNumeric,DeliveryAddress,FirstSubmissionDate,FolioNumber,IsPermitFinal,IsPrivateProvider,IssuedDate,Latitude,Longitude,Miami21Zone,NewAdditionCost,PermitNumber,PlanAcceptedDate,PlanCreatedDate,ProcessNumber,PropertyType,RemodelingCost,RemSQFT,RequiredCertificate,ScopeofWork,Statusdate,TotalCost,TotalDaysInPlanReview,TotalDaysInPlanReviewNumeric,TotalSQFT,WorkItems,ObjectId,geometry
48027,48027,NaN,BD19009714,NaT,None,Final,All Inspections are finalized.,C40,2019-10-05 02:31:00,,None,None,NaN,113 Day(s) 2 Hour(s) 13 Minute(s),113.0,99 SW 7 ST,2019-06-13 08:20:00+00:00,102050301090,YES,False,2019-10-05 02:31:00+00:00,25.767397,-80.195326,T6-48B-O,None,BD19009714001RC001,2019-06-13 08:15:00+00:00,2019-05-21 09:18:00+00:00,BD19009714001,Commercial,None,NaN,,40 YEAR AND OLDER RECERTIFICATION,2019-10-05 02:31:00+00:00,None,113 Day(s) 18 Hour(s) 16 Minute(s),113.0,NaN,40 YEAR AND OLDER RECERTIFICATION,48028,POINT (-80.19533 25.76740)
103275,103109,0.0,BD21025035,2022-12-01 15:33:00+00:00,None,Final,All Inspections are finalized.,None,NaT,5225 SW 90TH AVE,COOPER CITY,MILLER & MYERS GROUP LLC,33328.0,None,NaN,99 SW 7 ST,NaT,102050301090,YES,False,2022-05-10 16:09:00+00:00,25.767397,-80.195326,T6-48B-O,0,BD21025035001B001,2021-10-26 11:43:00+00:00,2021-10-25 09:53:00+00:00,BD21025035001,Commercial,0,0.0,,DEMOLITION,2022-12-01 15:33:00+00:00,105400,196 Day(s) 4 Hour(s) 26 Minute(s),196.0,0.0,DEMOLITION - TOTAL,103276,POINT (-80.19533 25.76740)
103383,103110,0.0,BD21025035,NaT,None,Final,All Inspections are finalized.,None,NaT,1329 SHOTGUN RD,SUNRISE,RIGHT WAY PLUMBING CO INC,33326.0,None,NaN,99 SW 7 ST,NaT,102050301090,YES,False,2022-10-25 16:47:00+00:00,25.767397,-80.195326,T6-48B-O,0,BD21025035001P001,2021-10-26 11:43:00+00:00,2021-10-25 09:53:00+00:00,BD21025035001,Commercial,0,0.0,,DEMOLITION,2022-11-17 10:08:00+00:00,500,364 Day(s) 5 Hour(s) 4 Minute(s),364.0,0.0,SEWER CAPPING,103384,POINT (-80.19533 25.76740)
125968,125693,0.0,BD23001839,NaT,None,Active,Permit is Active. Ready for Inspections.,None,NaT,2322 SW 58 TER,HOLLYWOOD,SUNSHINE TENTS & EVENTS RENTALS LLC,33023.0,None,NaN,99 SW 7 ST,NaT,102050301090,NO,False,2023-01-31 09:32:00+00:00,25.767397,-80.195326,T6-48B-O,None,BD23001839001SE001,2023-01-30 12:32:00+00:00,2023-01-25 16:24:00+00:00,BD23001839001,Commercial,None,0.0,,SPECIAL EVENTS,2023-01-31 09:32:00+00:00,2400,0 Day(s) 21 Hour(s) 0 Minute(s),0.0,0.0,TEMPORARY STRUCTURES: TENTS,125969,POINT (-80.19533 25.76740)


In [88]:
joined_permits_final.loc[joined_permits_final['FolioNumber'] == 102050301090]

,ID,AdditionSQFT,ApplicationNumber,BuildingFinalLastInspDate,BuildingFinalLastInspResult,BuildingPermitStatusDescription,BuildingPermitStatusReasonDescr,Certificatecode,Certificatedate,CompanyAddress,CompanyCity,CompanyName,CompanyZip,DaysInCity,DaysInCityNumeric,DeliveryAddress,FirstSubmissionDate,FolioNumber,IsPermitFinal,IsPrivateProvider,IssuedDate,Latitude,Longitude,Miami21Zone,NewAdditionCost,PermitNumber,PlanAcceptedDate,PlanCreatedDate,ProcessNumber,PropertyType,RemodelingCost,RemSQFT,RequiredCertificate,ScopeofWork,Statusdate,TotalCost,TotalDaysInPlanReview,TotalDaysInPlanReviewNumeric,TotalSQFT,WorkItems,ObjectId,geometry,index_right,Name,Description


## Make Map

In [41]:
def popup_html(row):
    address = row['DeliveryAddress']
    FolioNumber = row['FolioNumber']
    permit_status = row['BuildingPermitStatusDescription']
    permit_status_reason = row['BuildingPermitStatusReasonDescr']
    IssuedDate = row['IssuedDate']
    days_in_review = row['TotalDaysInPlanReview']
    TotalSQFT = row['TotalSQFT']
    NewAdditionCost = row['NewAdditionCost']
    PermitNumber = row['PermitNumber']
    ApplicationNumber = row['ApplicationNumber']
    geometry = row['geometry']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Folio: </strong>{}'''.format(FolioNumber) + '''<br>
    <strong>Permit Status: </strong>{}'''.format(permit_status) + '''<br>
    <strong>Reason for Status: </strong>{}'''.format(permit_status_reason) + '''<br>
    <strong>Issue Date: </strong>{}'''.format(IssuedDate) + '''<br>
    <strong>Days in review: </strong>{}'''.format(days_in_review) + '''<br>
    <strong>TotalSQFT: </strong>{}'''.format(TotalSQFT) + '''<br>
    <strong>New Addition Cost: </strong>{}'''.format(NewAdditionCost) + '''<br>
    <strong>Permit Number: </strong>{}'''.format(PermitNumber) + '''<br>
    <strong>Application Number: </strong>{}'''.format(ApplicationNumber) + '''<br>
    <strong>Location: </strong>{}'''.format(geometry) + '''<br>
    </html>
    '''
    return html

In [14]:
### Pop up for County Level Data ###

# def popup_html(row):
#     address = row['ADDRESS']
#     sdn_address = row['STNDADDR']
#     folio = row['FOLIO']
#     type_ = row['TYPE']
#     issue_date = row['ISSUDATE']
#     geometry = row['geometry']
    
#     html = '''<!DOCTYPE html>
#     <html>
#     <strong>Address: </strong>{}'''.format(address) + '''<br>
#     <strong>Standard Address: </strong>{}'''.format(sdn_address) + '''<br>
#     <strong>Folio: </strong>{}'''.format(folio) + '''<br>
#     <strong>Type: </strong>{}'''.format(type_) + '''<br>
#     <strong>Issue Date: </strong>{}'''.format(issue_date) + '''<br>
#     <strong>Location: </strong>{}'''.format(geometry) + '''<br>
#     </html>
#     '''
#     return html

In [42]:
# Create a map centered on Miami
m = folium.Map(location=[25.7617, -80.1918], zoom_start=12)

for i, r in greater_dtown_miami.iterrows():
    try:
      sim_geo = gpd.GeoSeries(r['geometry'])
      geo_j = sim_geo.to_json()
      geo_j = folium.GeoJson(data=geo_j)
      folium.Tooltip(r['Name']).add_to(geo_j)
      geo_j.add_to(m)
    except Exception as e:
      print(e)
      pass

# Add the points to the map
for index, row in joined_permits_final.iterrows():
    folium.Marker(location=[row['geometry'].y, row['geometry'].x],popup=folium.Popup(popup_html(row),max_width=400)).add_to(m)

# Display the map
m

In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [44]:
m.save('index.html')

In [45]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Greater_Miami_Projects


In [47]:
joined_permits_final.to_csv('Greater_Dtown_Miami_Permits.csv')